<a href="https://colab.research.google.com/github/amyush/Audio-steganography/blob/main/Audio_steganography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setting up the environment**

In [24]:
# Cloning the git repository to read the files
!git clone https://github.com/amyush/Audio-steganography.git

fatal: destination path 'Audio-steganography' already exists and is not an empty directory.


In [25]:
import numpy as np
import pandas as pd
import cv2
from google.colab.patches import cv2
import wave

# **Custom Functions**

In [26]:
# Function to convert a string to its binary representation
def stringToBinary(a):
  l=[]
  res = ''.join(format(ord(i), '08b') for i in a)
  return res

# Function to convert a binary number to its respective string
def binaryToString(a):
  return chr(binaryToInt(a))

# Function to convert a integer to its binary representation
def integerToBinary(a):
  return int(bin(a)[2:])

# Function to convert a binary to its integer value
def binaryToInt(binary): 
    int_val, i, n = 0, 0, 0
    while(binary != 0): 
        a = binary % 10
        int_val = int_val + a * pow(2, i) 
        binary = binary//10
        i += 1
    return int_val

# **Steganography**

## **Hiding the data**

In [35]:
# Function to encode the message within an audio file
def encodeMessage(filename, key, message):
    # Base path of the code
    audioPath = "Audio-steganography/"

    # Reading the audio file as read-only
    audioFile = wave.open(audioPath + filename, 'rb')
    
    # Extract audio frame as byte array
    audioBytes = bytearray(list(audioFile.readframes(audioFile.getnframes())))
    
    # Declaring a delimeter
    delimeter = "#@#@#@"
    length = len(audioBytes)

    # Converting key and message to its binary representation
    # Here, custom function is used for conversion.
    binKey = stringToBinary(key)
    binMsg = stringToBinary(message + delimeter)
    lenBinKey = len(binKey)
    lenBinMsg = len(binMsg)

    # Pointer to keep track of current key bit
    index = -1
    # Pointer to keep track of current message bit
    count = 0

    # New audio file where the message is encoded
    encodedAudioBytes = audioBytes

    # Hiding the message in the audio file
    for j in range(0, length):
        index = index + 1;

        # Looping over key bits using modulus function to accomodate any length of message
        if((int)(binKey[index%lenBinKey]) == 1):
            # If the count is less, that means not all message is yet encoded
            if(count < lenBinMsg):

                # Storing the current frame of the file in binary
                currentFrame = integerToBinary(audioBytes[j])
                
                # Changing the last bit of the current frame according to the message current bit
                modifiedFrame = ((int)(currentFrame/10)*10) + (int)(binMsg[count])

                # Storing the modified frame in the audio
                encodedAudioBytes[j] = binaryToInt(modifiedFrame);

                # Increasing the pointer to the message bit
                count = count + 1;
            # Else, all the message alongwith delimeter is encoded. Thus, breaking out of loop
            else:
                break;

    # Coverting the encoded audio file to bytes
    encodedAudioBytes = bytes(encodedAudioBytes)
    # Writing the audio file to the base path
    newaudio = wave.open(audioPath + "encodedAudio.wav", "wb")
    # Setting the params of the original file to the encoded file
    newaudio.setparams(audioFile.getparams())
    # Writing frames of original audio to the encoded audio file
    newaudio.writeframes(encodedAudioBytes)

## **Extracting the data**

In [36]:
# Function to extract the message from an audio file
def extractMessage(key):
    # Base path of the code
    audioPath ="Audio-steganography/"

    # Reading the audio file as read-only
    audioFile = wave.open(audioPath + 'encodedAudio.wav', 'rb')

    # Extract audio frame as byte array
    audioBytes = bytearray(list(audioFile.readframes(audioFile.getnframes())))
    length = len(audioBytes)

    # Declaring a delimeter
    delimeter = "#@#@#@"

    # Converting key to its binary representation
    # Here, custom function is used for conversion.
    binKey = stringToBinary(key)
    lenBinKey = len(binKey)

    # Pointer to keep track of current key bit
    index = -1

    # Each character is of 8 bytes.
    # Counter to keep track of length of current frame value
    extractedBitLength = 0

    # Variable to hold the extracted bits
    extractedBit = 0

    # Extracted message
    extractedMessage = ""

    # Hiding the message in the audio file
    for j in range(0, length):
        index += 1

        # Looping over key bits using modulus function to accomodate any length of message
        if((int)(binKey[index%lenBinKey]) == 1):
            
            # Converting the current frame to binary
            currentFrame = integerToBinary(audioBytes[j])

            # Updating the extracted bit
            extractedBit = extractedBit * 10 + currentFrame%10
            extractedBitLength += 1

            # When the length of the extracted bit is 8, convert the bits to string.
            # This gives 1 extracted character.
            if(extractedBitLength == 8):

                # Converting the extracted bit to its character representation, and adding to the extracted message.
                extractedMessage += binaryToString(extractedBit)

                # Re-set the length of the extracted bit length to 0
                extractedBitLength = 0

                # Re-set the extracted bit to 0 for the next character
                extractedBit = 0

                # If the extracted message has the delimeter, it means the message has been extracted completely.
                # Thus we return the message
                if("#@#@#@" in extractedMessage):
                    # Return the message by leaving the last 6 characters of the delimeter
                    return extractedMessage[:-6]
              

# **Results**

In [37]:
key = 'apple'
message = 'awdoajiwd ijawodjajwiojd'
encodeMessage('cover_audio.wav', key, message)

In [38]:
extractedMessage = extractMessage(key)

print('encodedMessage = {}'.format(message))

print('extractedMessage = {}'.format(extractedMessage))

encodedMessage = awdoajiwd ijawodjajwiojd
extractedMessage = awdoajiwd ijawodjajwiojd
